In [ ]:
# !pip install --quiet --pre --upgrade dgl-cu101
# !pip install --quiet torch==1.6.0

In [ ]:
import argparse
import itertools
import os
import numpy as np
from numpy import save,load,savetxt,loadtxt,savez_compressed
from sklearn.metrics import roc_auc_score, f1_score,average_precision_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc as auc_score
import pandas as pd
import scipy.sparse as sp
import time
from tqdm import tqdm, tqdm_notebook,tnrange
tqdm.pandas(position=0, leave=True)
import math 

import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import dgl.function as fn
from dgl.ops import edge_softmax

from functools import partial
import seaborn as sns
import pickle
import random
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
import warnings
warnings.filterwarnings('ignore')
import utils
import tsne_func
print("torch version is {}".format(th.__version__))
print("DGL version is {}".format(dgl.__version__))

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    th.manual_seed(seed)
    th.cuda.manual_seed_all(seed)
    th.backends.cudnn.deterministic = True
    th.backends.cudnn.benchmark = False
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(101)

In [ ]:
KG_dir="/workspace/cjiang/eagle_project/CAP_graph/dataset/"
start=time.time()
with open(os.path.join(KG_dir,'CAP_Graph'), 'rb') as f:
    G, node_labels = pickle.load(f)
end=time.time()
print("It took {:0.4f} seconds to load graph".format(end-start))

In [ ]:
# utils.graph_show(G)

In [ ]:
usaanr_feat=[]
for key, scheme in G.node_attr_schemes(ntype="usaanr").items():
    usaanr_feat.append(key)
usaanr_feat=[x for x in usaanr_feat if x not in ['type','usaanr','train_mask','val_mask','test_mask']]

print()
print("The features associated with USAA Member are\n ")
for i in usaanr_feat:
    print(i)

In [ ]:
## USAA Members Features Embedding
class USAANR_Embedding(nn.Module):
    def __init__(self,G,feature_size):
        super(USAANR_Embedding,self).__init__()
        self.G=G
        self.feature_size=feature_size
        ## Embedding matrices for features of nodes.
        self.emb = nn.ModuleDict()
        
        
        for i,col in enumerate(usaanr_feat):
            self.emb[col]=nn.Embedding(G.nodes['usaanr'].data[col].max().item()+1, feature_size)
        ## Embedding for the node of House Properties
#         self.node_emb=nn.Embedding(G.filter_nodes(lambda nodes: (nodes.data['node_type'] == 0).squeeze(1),\
#                                        ntype='House_Properties').max().item() + 1, feature_size)
        self.node_emb=nn.Embedding(2, feature_size)
    
    def forward(self,nid):
        nid=nid.to("cpu")
        h=self.node_emb(self.G.nodes['usaanr'].data['type'][nid].squeeze(1).to(device))
        extra_repr=[]
        for i,col in enumerate(usaanr_feat):
            ndata=self.G.nodes['usaanr'].data[col]
            extra_repr.append(self.emb[col](ndata[nid].to(device)).squeeze(1))
        return h + th.stack(extra_repr, 0).sum(0)

## zipcode Embedding
class Zipcode_Embedding(nn.Module):
    def __init__(self,G,feature_size):
        super(Zipcode_Embedding,self).__init__()
        self.G=G
        self.feature_size=feature_size
        
        ## Embedding for the node of zipcode.
#         self.node_emb=nn.Embedding(G.filter_nodes(lambda nodes: (nodes.data['type'] == 1).squeeze(1),\
#                                        ntype='zipcode').max().item() + 1, feature_size)
        self.node_emb=nn.Embedding(2, feature_size)
    
    def forward(self,nid):
        nid=nid.to("cpu")
        h=self.node_emb(self.G.nodes['zipcode'].data['type'][nid].squeeze(1).to(device))
        return h 

class HGTLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 node_dict,
                 edge_dict,
                 n_heads,
                 dropout = 0.2,
                 use_norm = False):
        super(HGTLayer, self).__init__()

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.node_dict     = node_dict
        self.edge_dict     = edge_dict
        self.num_types     = len(node_dict)
        self.num_relations = len(edge_dict)
        self.total_rel     = self.num_types * self.num_relations * self.num_types
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None

        self.k_linears   = nn.ModuleList()
        self.q_linears   = nn.ModuleList()
        self.v_linears   = nn.ModuleList()
        self.a_linears   = nn.ModuleList()
        self.norms       = nn.ModuleList()
        self.use_norm    = use_norm

        for t in range(self.num_types):
            self.k_linears.append(nn.Linear(in_dim,   out_dim))
            self.q_linears.append(nn.Linear(in_dim,   out_dim))
            self.v_linears.append(nn.Linear(in_dim,   out_dim))
            self.a_linears.append(nn.Linear(out_dim,  out_dim))
            if use_norm:
                self.norms.append(nn.LayerNorm(out_dim))

        self.relation_pri   = nn.Parameter(th.ones(self.num_relations, self.n_heads))
        self.relation_att   = nn.Parameter(th.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg   = nn.Parameter(th.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.skip           = nn.Parameter(th.ones(self.num_types))
        self.drop           = nn.Dropout(dropout)

        nn.init.xavier_uniform_(self.relation_att)
        nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, sg, h):
        with sg.local_scope():
            node_dict, edge_dict = self.node_dict, self.edge_dict
            
            #### remove the edges where the number of edges==0
            etypes_with_nonzero_edges = [etype for etype in sg.etypes if sg.num_edges(etype) > 0]
            sg=sg.edge_type_subgraph(etypes_with_nonzero_edges) 
            
            for srctype, etype, dsttype in sg.canonical_etypes:
                sub_graph = sg[srctype, etype, dsttype]
                
                if sub_graph.num_edges()==0:
                    continue
                else:   
                    k_linear = self.k_linears[node_dict[srctype]]
                    v_linear = self.v_linears[node_dict[srctype]]
                    q_linear = self.q_linears[node_dict[dsttype]]

                    k = k_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                    v = v_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                    q = q_linear(h[dsttype][0:sg.num_dst_nodes()]).view(-1, self.n_heads, self.d_k)

                    e_id = self.edge_dict[etype]

                    relation_att = self.relation_att[e_id]
                    relation_pri = self.relation_pri[e_id]
                    relation_msg = self.relation_msg[e_id]

                    k = th.einsum("bij,ijk->bik", k, relation_att)
                    v = th.einsum("bij,ijk->bik", v, relation_msg)

                    sub_graph.srcdata['k'] = k
                    sub_graph.dstdata['q'] = q
                    sub_graph.srcdata['v'] = v

                    sub_graph.apply_edges(fn.v_dot_u('q', 'k', 't'))
                    attn_score = sub_graph.edata.pop('t').sum(-1) * relation_pri / self.sqrt_dk
                    attn_score = edge_softmax(sub_graph, attn_score, norm_by='dst')

                    sub_graph.edata['t'] = attn_score.unsqueeze(-1)

            sg.multi_update_all({etype : (fn.u_mul_e('v', 't', 'm'), fn.sum('m', 't')) \
                                for etype in edge_dict}, cross_reducer = 'mean')

            new_h = {}
            for ntype in sg.ntypes:
                '''
                    Step 3: Target-specific Aggregation
                    x = norm( W[node_type] * gelu( Agg(x) ) + x )
                '''
                n_id = node_dict[ntype]
                alpha = th.sigmoid(self.skip[n_id])
                t = sg.nodes[ntype].data['t'].view(-1, self.out_dim)
                trans_out = self.drop(self.a_linears[n_id](t))
                trans_out = trans_out * alpha + h[ntype] * (1-alpha)
                if self.use_norm:
                    new_h[ntype] = self.norms[n_id](trans_out)
                else:
                    new_h[ntype] = trans_out
            return new_h
        
class HGT(nn.Module):
    def __init__(self, G, node_dict, edge_dict, in_feat, h_dim, out_feat, n_layers, n_heads, use_norm = True):
        super(HGT, self).__init__()
        self.G=G
        self.node_dict = node_dict
        self.edge_dict = edge_dict
        self.gcs = nn.ModuleList()
        self.in_feat = in_feat
        self.h_dim = h_dim
        self.out_feat = out_feat
        self.n_layers = n_layers
        self.adapt_ws  = nn.ModuleList()
        for t in range(len(node_dict)):
            self.adapt_ws.append(nn.Linear(in_feat,   h_dim))
        for _ in range(n_layers):
            self.gcs.append(HGTLayer(h_dim, h_dim, node_dict, edge_dict, n_heads, use_norm = use_norm))
        self.out = nn.Linear(h_dim, out_feat)
        
        self.node_embed=nn.ModuleDict()
        self.node_embed['usaanr'] = USAANR_Embedding(self.G,self.in_feat)
        self.node_embed['zipcode'] = Zipcode_Embedding(self.G,self.in_feat)
           
    def forward(self, input_nodes, blocks=None):
        H = {}
        for ntype, nid in input_nodes.items():
            nid = input_nodes[ntype]
            H[ntype] = F.gelu(self.adapt_ws[self.node_dict[ntype]](self.node_embed[ntype](nid)))
        
        if blocks is None:
            for layer in self.gcs:
                H = layer(self.G, H)
        else:
            for layer, block in zip(self.gcs, blocks):
                H = layer(block, H)
        return self.out(H["usaanr"])       
        



In [ ]:
def evaluate(model, loader, labels, category, device):
    model.eval()
    total_loss = 0
    total_acc = 0
#     total_precision=0
#     total_recall=0
#     total_fscore=0
    total_auc=0
#     total_pr_auc=0
    
    count = 0
    count_loss=0
    
    for input_nodes, seeds, blocks in tqdm(loader, position=0, leave=True):
        blocks = [blk.to(device) for blk in blocks]
        seeds = seeds[category]
        input_nodes={k : e.to(device) for k, e in input_nodes.items()}
        lbl = labels[seeds].to(device)
        logits,h = model(input_nodes,blocks)
        loss = F.cross_entropy(logits, lbl.squeeze(1).to(device))
#         loss = F.cross_entropy(logits, lbl.squeeze(1),weight=th.Tensor([1,args.weight]).to(device))
        acc = th.sum(logits.argmax(dim=1) == lbl.squeeze(1)).item() / logits.shape[0]
#         precision, recall, fscore, support = score(lbl.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy())

        tempt=lbl.detach().cpu().numpy()
        labels_train_one_hot=np.zeros(shape=(tempt.shape[0],8),dtype=np.float32)
        labels_train_one_hot[np.arange(tempt.shape[0]),np.array([ele.item() for ele in tempt])]=1
        auc = roc_auc_score(labels_train_one_hot.ravel(), th.sigmoid(logits).detach().cpu().numpy().ravel())
        
#         auc = roc_auc_score(lbl.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
#         prec,rec,_ = precision_recall_curve(lbl.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
#         pr_auc=auc_score(rec,prec)
        
        total_loss += loss.item() * len(seeds) 
        total_acc += acc
#         total_precision += precision[1]
#         total_recall += recall[1]
#         total_fscore += fscore[1]
        total_auc += auc
#         total_pr_auc += pr_auc
        count += 1
        count_loss += len(seeds)
    
    ACCURACY=total_acc / count
    LOSS=total_loss / count_loss
#     PRECISION=total_precision/count
#     RECALL=total_recall/count
#     F1_SCORE=total_fscore/count
    AUC=total_auc/count
#     PR_AUC=total_pr_auc/count
    
    return ACCURACY, LOSS, AUC

#### Create subgraph for the purpose of preliminary test

In [ ]:
# dict_nodes={"usaanr":th.arange(G.num_nodes('usaanr'))[0:1000],'zipcode':th.arange(G.num_nodes('zipcode'))[0:100]}
# sg=dgl.node_subgraph(G,dict_nodes)

dict_edges={}
for etype in G.etypes:
    dict_edges[etype]=th.arange(G.num_edges(etype))[0:5000]
sg=dgl.edge_subgraph(G,dict_edges)

In [ ]:
utils.graph_show(sg)

In [ ]:
sg.nodes['usaanr'].data["_ID"].numpy().shape,node_labels.shape, node_labels[sg.nodes['usaanr'].data["_ID"]].shape

#### settings

In [ ]:
parser = argparse.ArgumentParser(description='HGT')
parser.add_argument("--dropout", type=float, default=0,
        help="dropout probability")
parser.add_argument("--h_dim", type=int, default=128,
        help="number of hidden units")
parser.add_argument("--out_dim", type=int, default=1,
        help="output dimension")
parser.add_argument("--gpu", type=int, default=0,
        help="gpu")
parser.add_argument("--lr", type=float, default=1e-5,
        help="learning rate")
parser.add_argument('--clip',    type=int, default=1.0) 
parser.add_argument('--max_lr',  type=float, default=1e-3) 

parser.add_argument("--num_layers", type=int, default=1,
        help="number of propagation rounds")
parser.add_argument("-e", "--n_epochs", type=int, default=1,
        help="number of training epochs")
parser.add_argument("--model_path", type=str, default="/workspace/cjiang/eagle_project/CAP_graph/hgt_model_param.pt",
        help='path for save the model')
parser.add_argument("--l2norm", type=float, default=0,
        help="l2 norm coef")

parser.add_argument("--batch-size", type=int, default=1024,
        help="Mini-batch size. If -1, use full graph training.")
parser.add_argument("--num_mini_batch", type=int, default=8,
        help="Number of minibatch.")
parser.add_argument("--fanout", type=int, default=None,
        help="Fan-out of neighbor sampling.")

parser.add_argument("--seed",  type=int,default=101,
        help="random seed for np.random.seed, torch.manual_seed and torch.cuda.manual_seed.")

parser.add_argument("--num_worker",  type=int,default=4,  
        help="number of worker for neighbor sampling") 

args,unknown=parser.parse_known_args()

args.num_layers=1
args.dropout=0.2
args.lr=1e-3
args.l2norm=1e-3
args.n_epochs=1
args.h_dim=64
args.batch_size=1024
print(args)

#### setting up training, validation and test set

In [ ]:
num_rels=len(G.etypes)
LABEL=th.tensor(node_labels[sg.nodes['usaanr'].data["_ID"]]).long()
labels, count=th.unique(LABEL,return_counts=True)
num_classes=labels.shape[0]
pd.DataFrame({"label_class":labels, "count":count}).style.format({'count':'{:,}'})

In [ ]:
train_mask=sg.nodes["usaanr"].data.pop('train_mask')
val_mask=sg.nodes["usaanr"].data.pop('val_mask')
test_mask=sg.nodes["usaanr"].data.pop('test_mask')

train_idx=th.nonzero(train_mask.squeeze(1)).numpy()
val_idx=th.nonzero(val_mask.squeeze(1)).numpy()
test_idx=th.nonzero(test_mask.squeeze(1)).numpy()

train_idx=th.from_numpy(train_idx).squeeze(1)    
val_idx=th.from_numpy(val_idx).squeeze(1)    
test_idx=th.from_numpy(test_idx).squeeze(1)

train_label=LABEL[train_idx]
val_label=LABEL[val_idx]
test_label=LABEL[test_idx]

print('{:<15} {:<10,}'.format("Training set",train_idx.shape[0]))
print('{:<15} {:<10,}'.format("validation set",val_idx.shape[0]))
print('{:<15} {:<10,}'.format("test set",test_idx.shape[0]))

In [ ]:
assert train_idx.shape[0]+val_idx.shape[0]+test_idx.shape[0] == sg.nodes['usaanr'].data["_ID"].shape[0]

In [ ]:
##### check cuda
device="cpu"
use_cuda=args.gpu>=0 and th.cuda.is_available()
if use_cuda:
    th.cuda.set_device(args.gpu)
    device='cuda:%d' % args.gpu
print(device)

In [ ]:
node_dict = {}
edge_dict = {}
for ntype in sg.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in sg.etypes:
    edge_dict[etype] = len(edge_dict)
    sg.edges[etype].data['id'] = th.ones(sg.number_of_edges(etype), dtype=th.long) * edge_dict[etype] 

print(node_dict)
print(edge_dict)

In [ ]:
for etype in sg.etypes:
    for key,val in sg.edges[etype].data.items():
        if key=="id":
            print("{:<15}{:<10}{:<10}".format(etype,key,th.unique(val).item()))

In [ ]:
# create model

model = HGT(sg,
            node_dict, 
            edge_dict,
            in_feat=args.h_dim,
            h_dim=args.h_dim,
            out_feat=num_classes,
            n_layers=2,
            n_heads=4,
            use_norm = True)

if use_cuda:
    model.cuda()

model.gcs

In [ ]:
# optimizer = th.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2norm)
optimizer = th.optim.AdamW(model.parameters())
scheduler = th.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.n_epochs, max_lr = args.max_lr)

In [ ]:
# train sampler
sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
train_loader = dgl.dataloading.NodeDataLoader(
    sg, {'usaanr': train_idx}, sampler,
    batch_size=args.batch_size, shuffle=True, num_workers=args.num_worker)
# validation sampler
# we do not use full neighbor to save computation resources
val_sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
val_loader = dgl.dataloading.NodeDataLoader(
    sg, {'usaanr': val_idx}, val_sampler,
    batch_size=args.batch_size, shuffle=False, num_workers=args.num_worker)

test_sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
test_loader = dgl.dataloading.NodeDataLoader(
    sg, {'usaanr': test_idx}, test_sampler,
    batch_size=args.batch_size, shuffle=False, num_workers=args.num_worker)

print("The number of minibatch in training set is {:,}".format(len(train_loader)))
print("The number of minibatch in validation set is {:,}".format(len(val_loader)))
print("The number of minibatch in test set is {:,}".format(len(test_loader)))

In [ ]:
print("The total # of parameter is {:,}".format(sum([p.nelement() for p in model.parameters()]) ) )

In [ ]:
param_dict={n: p.nelement() for n, p in model.named_parameters()}
for i,j in param_dict.items():
    print("{:<70}{:<15,}".format(i,j))

In [ ]:
# %pdb
LOSS_EPOCH=[]
LABEL_TRAIN=[]
ACC_EPOCH=[]
# training loop
print("start training...")
dur = []
total_loss=0
losses=[]

# th.manual_seed(args.seed)
# th.cuda.manual_seed(args.seed)
# th.cuda.manual_seed_all(args.seed)
# np.ranom.seed(args.seed)
# random.seed(args.seed)
# th.backends.cudnn.deterministic=True

for epoch in tqdm(range(0,args.n_epochs)):
    
    model.train()
    IDX=[]
    H=[]
    
    #====================================#
    #            Traning                 #
    #====================================#
    print("")
    print("========= Epoch {:} /{:}".format(epoch+1,args.n_epochs))
    print("Training...")
    t0 = time.time()
    for step, (input_nodes, seeds, blocks) in enumerate(train_loader):
        blocks = [blk.to(device) for blk in blocks]
        seeds=seeds["usaanr"].to(device)
        labels_train=LABEL[seeds]       
        labels_train = labels_train.to(device)
        input_nodes={k : e.to(device) for k, e in input_nodes.items()}
        logits,h = model(input_nodes,blocks)
        optimizer.zero_grad()
#         loss = F.cross_entropy(logits, labels_train.squeeze(1),weight=th.Tensor([1,args.weight]).to(device))
        loss = F.cross_entropy(logits, labels_train.squeeze(1).to(device))
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        train_acc = th.sum(logits.argmax(dim=1) == labels_train.squeeze(1)).item() / len(seeds)
#         precision, recall, fscore, support = score(labels_train.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy())

        tempt=labels_train.detach().cpu().numpy()
        labels_train_one_hot=np.zeros(shape=(tempt.shape[0],8),dtype=np.float32)
        labels_train_one_hot[np.arange(tempt.shape[0]),np.array([ele.item() for ele in tempt])]=1
        train_auc = roc_auc_score(labels_train_one_hot.ravel(), th.sigmoid(logits).detach().cpu().numpy().ravel())
#         train_auc = roc_auc_score(labels_train.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
        
#         prec,rec,_ = precision_recall_curve(labels_train.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
#         train_pr_auc=auc_score(rec,prec)

        IDX.extend(seeds.detach().cpu().numpy().tolist())
        H.extend(h["usaanr"].detach().cpu().numpy().tolist())
        
        if step%(len(train_loader)//10)==0 and not step==0:

            t1 = time.time()
            elapsed=utils.format_time(t1-t0)
            print("Batch {:} of {:} | Loss {:.3f}  | Accuracy {:.2%} | ROC_AUC {:.2%}  | Elapsed: {:}".\
                  format(step,len(train_loader),np.mean(losses[-10:]),train_acc,train_auc,elapsed))    
    
    LOSS_EPOCH.append(loss)

    LABEL_TRAIN.append(LABEL[blocks[-1].nodes["usaanr"].data[dgl.NID].cpu().numpy()])
#     ACC_EPOCH.append(train_acc)

    model.eval()
    print()
    print("")
    print("Running Validation on training set")
    print("")
    
    
    acc_train, loss_train, train_auc = evaluate(model, train_loader,  LABEL, "usaanr", device)
    
    t2=time.time()
    
    print("loss: {:.3f} |  Accuracy {:.2%} | ROC_AUC: {:.2%}  | Elapsed: {:}"\
      .format(loss_train, acc_train,  train_auc, utils.format_time(t2-t1)))

    #====================================#
    #            Validation-set          #
    #====================================#
    
    model.eval()
    print()
    print("")
    print("Running Validation")
    print("")
    acc_val, loss_val, val_auc = evaluate(model, val_loader,  LABEL, "usaanr", device)
    
    t3=time.time()
    
    print("loss: {:.3f} |  Accuracy {:.2%}  | ROC_AUC: {:.2%} | Elapsed: {:}"\
      .format(loss_val, acc_val, val_auc, utils.format_time(t3-t2)))
    
if args.model_path is not None:
    th.save(model.state_dict(), args.model_path)
    
#====================================#
#            Test-set                #
#====================================#
print()
print("")
print("Running Validation in Test Dataset")
print("")
model.eval()

acc_test, loss_test, test_auc= evaluate(model, test_loader,  LABEL, "usaanr", device)

t3=time.time()
print("loss: {:.3f} |  Accuracy {:.2%} | ROC_AUC: {:.2%} | Elapsed: {:}"\
      .format(loss_test, acc_test, test_auc, utils.format_time(t3-t2)))

In [ ]:
input_nodes

In [ ]:
blocks[0]

In [ ]:
seeds

In [ ]:
logits,h=model(input_nodes,blocks)

In [ ]:
adapt_ws  = nn.ModuleList()
for t in range(len(node_dict)):
    adapt_ws.append(nn.Linear(args.h_dim,   args.h_dim))
adapt_ws

In [ ]:
n_heads=4
gcs = nn.ModuleList()
for _ in range(args.num_layers):
    gcs.append(HGTLayer(args.h_dim, args.h_dim, node_dict, edge_dict, n_heads, use_norm = True))

In [ ]:
gcs

In [ ]:
H = {}
for ntype, nid in input_nodes.items():
    nid = input_nodes[ntype]
    H[ntype] = F.gelu(adapt_ws[nid](node_embed[ntype](nid)))

In [ ]:
node_embed=nn.ModuleDict()
node_embed['usaanr'] = USAANR_Embedding(sg,args.h_dim)
node_embed['zipcode'] = Zipcode_Embedding(sg,args.h_dim)
node_embed[ntype](nid).shape

#### sparsity rate of embedding vector

In [ ]:
H=np.array(H)
non_zero=np.count_nonzero(H)
total_val=np.product(H.shape)
sparsity=(total_val-non_zero)/total_val
density=non_zero/total_val
print("sparsity rate is {:.2%}".format(sparsity))
print("density rate is {:.2%}".format(density))
print("embedding vector shape is {}".format(H.shape))

In [ ]:
IDX_train=np.array(IDX)
H_train=np.array(H)
mask_train=np.array(['train']*len(IDX_train))

In [ ]:
IDX_val=[]
H_val=[]
for input_nodes, seeds, blocks in tqdm(val_loader, position=0, leave=True):
    blocks = [blk.to(device) for blk in blocks]
    seeds = seeds["usaanr"]
    input_nodes={k : e.to(device) for k, e in input_nodes.items()}
    model.eval()
    logits,h = model(input_nodes,blocks)
    IDX_val.extend(seeds.detach().cpu().numpy().tolist())
    H_val.extend(h["usaanr"].detach().cpu().numpy().tolist())
    
IDX_val=np.array(IDX_val)
H_val=np.array(H_val)
mask_val=np.array(['val']*len(IDX_val))

In [ ]:
IDX_test=[]
H_test=[]
for input_nodes, seeds, blocks in tqdm(test_loader, position=0, leave=True):
    blocks = [blk.to(device) for blk in blocks]
    seeds = seeds["usaanr"]
    input_nodes={k : e.to(device) for k, e in input_nodes.items()}
    model.eval()
    logits,h = model(input_nodes,blocks)
    IDX_test.extend(seeds.detach().cpu().numpy().tolist())
    H_test.extend(h["usaanr"].detach().cpu().numpy().tolist())
    
IDX_test=np.array(IDX_test)
H_test=np.array(H_test)
mask_test=np.array(['test']*len(IDX_test))

In [ ]:
IDX=np.concatenate((IDX_train,IDX_val, IDX_test))
H=np.concatenate((H_train,H_val, H_test))
mask=np.concatenate((mask_train,mask_val, mask_test))

_idx=IDX.argsort()  #### sort the node id from 0 to max_num, so that it can be matched with LABEL
embedding_vector=H[_idx]
mask=mask[_idx]

print("{:<30}{}".format("shape of embedding vector",embedding_vector.shape))

In [ ]:
%%time
savez_compressed("embedding_vector.npz",embedding_vector)
embedding_vector=load("embedding_vector.npz")['arr_0']

In [ ]:
# LABEL.shape, H.shape,IDX_train.shape,IDX_val.shape, IDX_test.shape

#### visualization of embedding vectors for different categories of USAA Members

In [ ]:
DF=pd.DataFrame({"IDX":_idx.tolist(), "MASK":mask.tolist(), "Product":LABEL.squeeze().tolist()})
prod_map={0:"No_Product",1:"Rental_only",2:"Home_only",3:"Home+Rental",4:"Auto_only",5:"Auto+Rental",6:"Auto+Home",7:"Auto+Home_Rental"}
DF['Product'] = list(map(prod_map.get, DF['Product']))

### the caterogry of "Home+Rental" is too rare, only 2767 in total
inx=np.where(DF["Product"].values!="Home+Rental")[0]
offset=np.where(DF["Product"].values=="Home+Rental")[0].shape[0]
DF=DF[DF["Product"]!="Home+Rental"]
DF["IDX"]=DF["IDX"]- offset ## since removing "Home+Rental", the index should be offset
embedding_vector=embedding_vector[inx]

N=1000

# sample_df=DF.groupby(['Product'], group_keys=False).apply(lambda x: x.sample(n=N))
# sample_id=sample_df["IDX"].values
# embedding_sample=embedding_vector[sample_id]
# prod_sample=DF['Product'].values[sample_id]

sample_df=DF.groupby(['MASK','Product'], group_keys=False).apply(lambda x: x.sample(n=N))
sample_id_train=sample_df[sample_df["MASK"]=="train"]["IDX"].values
sample_id_val=sample_df[sample_df["MASK"]=="val"]["IDX"].values
sample_id_test=sample_df[sample_df["MASK"]=="test"]["IDX"].values

embedding_sample_train=embedding_vector[sample_id_train]
embedding_sample_val=embedding_vector[sample_id_val]
embedding_sample_test=embedding_vector[sample_id_test]

prod_sample_train=DF['Product'].values[sample_id_train]
prod_sample_val=DF['Product'].values[sample_id_val]
prod_sample_test=DF['Product'].values[sample_id_test]


In [ ]:
a,b=np.unique(prod_sample_train,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
a,b=np.unique(prod_sample_val,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
a,b=np.unique(prod_sample_test,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
%%time
print("t-SNE dimension reduction for training embedding vector:")
print()
train_embedding = tsne_func.fit(embedding_sample_train)
print("t-SNE dimension reduction for validation embedding:")
print()
val_embedding = tsne_func.fit(embedding_sample_val)
print("t-SNE dimension reduction for test embedding:")
print()
test_embedding = tsne_func.fit(embedding_sample_test)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 8)
ax=sns.scatterplot(train_embedding[:,0], train_embedding[:,1], hue=prod_sample_train.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE(sample=1000) \n Training Set ",fontsize=15)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 2)
ax=sns.scatterplot(val_embedding[:,0], val_embedding[:,1], hue=prod_sample_val.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE (sample=1000) \n Validation Set ",fontsize=15)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 2)
ax=sns.scatterplot(test_embedding[:,0], test_embedding[:,1], hue=prod_sample_test.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE (sample=1000) \n Test Set ",fontsize=15)